In [3]:
import sys
from sqlalchemy import create_engine
import pandas as pd
import os
from os import path
import re
import pymysql
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
from multiprocessing import Pool
import yaml

with open('../config.yml') as f:
    config = yaml.load(f)

project_dir = config['PROJECT']
project_db_dir = config['DATABASE']

def imap_unordered_bar(func, args, number_of_elements, n_processes = 30):
    scores = []
    p = Pool(n_processes)

    with tqdm(total = number_of_elements) as pbar:
        for res in tqdm(p.imap_unordered(func, args, chunksize=n_processes)):
            scores += res
            pbar.update()

    pbar.close()
    p.close()
    p.join()

    return scores

engine = create_engine("mysql+pymysql://{user}:{pw}@{server}/{db}"
                       .format(server=config['MYSQL_HOST'], user=config['MYSQL_USER'], pw=config['MYSQL_PASSWD'], 
                               db=config['MYSQL_DB']))
con = pymysql.connect(host=config['MYSQL_HOST'], user=config['MYSQL_USER'], passwd=config['MYSQL_PASSWD'], 
                      db=config['MYSQL_DB'])

cur = con.cursor()

plot_dir = project_dir+'/tmp/clinvar/'
alignment_dir = project_dir+"/results/alignments/"

polarity = {
	'A': 'nonpolar',
    'B': '-',
    'Z': '-',
	'R': 'polar',
	'N': 'polar',
	'D': 'polar',
	'C': 'nonpolar',
	'E': 'polar',
	'Q': 'polar',
	'G': 'nonpolar',
	'H': 'polar',
	'I': 'nonpolar',
	'L': 'nonpolar',
	'K': 'polar',
	'M': 'nonpolar',
	'F': 'nonpolar',
	'P': 'nonpolar',
	'S': 'polar',
	'T': 'polar',
	'W': 'nonpolar',
	'Y': 'polar',
	'V': 'nonpolar',
	'U': 'polar',
	'-': '-'
}
def sqlize(significance_list):
	return "'"+ "', '".join(significance_list) + "'"

ncbi_to_transcript_id = pd.read_sql("SELECT gdb.db_id AS NCBI, gdb.convart_gene_id AS id, ncbi.meta_value AS gene_symbol "
								"FROM convart_gene_to_db AS gdb " +
								"LEFT JOIN ncbi_gene_meta AS ncbi ON ncbi.ncbi_gene_id = gdb.db_id WHERE gdb.db='NCBI' "+
								"AND ncbi.meta_key = 'gene_symbol' GROUP BY gdb.convart_gene_id", con)
ncbi_to_transcript_id.set_index('id', drop=False, inplace=True)


genes = pd.read_sql("SELECT DISTINCT g.id, fasta, sequence, m.id as msa_id "+
			"FROM convart_gene AS g "+
			"INNER JOIN msa_best_combination AS mb ON mb.convart_gene_id = g.id " +
			"INNER JOIN msa AS m ON m.id = mb.msa_id "
			"WHERE g.species_id='Homo sapiens'", con)


benign_list = sqlize(["Benign","Benign, association","Benign/Likely benign","Benign/Likely benign, Affects","Benign/Likely benign, association",
			"Benign/Likely benign, drug response","Benign/Likely benign, drug response, risk factor","Benign/Likely benign, other","Benign/Likely benign, protective",
			"Benign/Likely benign, protective, risk factor","Benign/Likely benign, risk factor","Benign, other","Benign, risk factor","Likely benign",
			"Likely benign, drug response, other","Likely benign, other","Likely benign, risk factor"])
vus_list = sqlize(["Uncertain significance","Uncertain significance, drug response","Uncertain significance, other","Uncertain significance, risk factor"])
pathogenic_list = sqlize(["Pathogenic","Pathogenic, Affects","Pathogenic, association, protective","Pathogenic, drug response","Pathogenic/Likely pathogenic",
"Pathogenic/Likely pathogenic, drug response","Pathogenic/Likely pathogenic, other","Pathogenic/Likely pathogenic, risk factor","Pathogenic, other",
"Pathogenic, other, risk factor","Pathogenic, protective","Pathogenic, risk factor","Likely pathogenic","Likely pathogenic, association",
"Likely pathogenic, drug response","Likely pathogenic, other","Likely pathogenic, risk factor"])

ptm_data = pd.read_sql("SELECT cdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions"
					+", COUNT(DISTINCT position) as variation_count FROM ptm INNER JOIN convart_gene_to_db AS cdb ON cdb.db_id=ptm.acc_id  WHERE position > 0 "
                    +"GROUP BY cdb.convart_gene_id", con)
clinvar_data = pd.read_sql("SELECT gdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions"
			+", COUNT(DISTINCT position) as variation_count FROM clinvar INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=np_id WHERE position > 0 "
                           +"GROUP BY gdb.convart_gene_id", con)
pathogenic_clinvar_data = pd.read_sql("SELECT gdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions, COUNT(DISTINCT position) "
			+"as variation_count FROM clinvar INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=np_id WHERE position > 0 AND clinical_significance IN ("+pathogenic_list+")"+
			" GROUP BY gdb.convart_gene_id", con)
vus_clinvar_data = pd.read_sql("SELECT gdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions, COUNT(DISTINCT position) "
			+"as variation_count FROM clinvar INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=np_id WHERE position > 0 AND clinical_significance IN ("+vus_list+")"+
			" GROUP BY gdb.convart_gene_id", con)
benign_clinvar_data = pd.read_sql("SELECT gdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions, COUNT(DISTINCT position) "
			+"as variation_count FROM clinvar INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=np_id WHERE position > 0 AND clinical_significance IN ("+benign_list+")"+
			" GROUP BY gdb.convart_gene_id", con)
gnomad_data = pd.read_sql("SELECT gdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions"
					+", COUNT(DISTINCT position) as variation_count FROM gnomad INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=canonical_transcript WHERE position > 0 "+
                     " GROUP BY gdb.convart_gene_id", con)

cosmic_data = pd.read_sql("SELECT gdb.convart_gene_id AS transcript_id, GROUP_CONCAT(DISTINCT position) as positions"
					+", COUNT(DISTINCT position) as variation_count FROM CosmicMutantExport INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=accession_number WHERE position > 0 "+
                     " GROUP BY gdb.convart_gene_id", con)
dbsnp_data = pd.read_sql("SELECT Feature AS transcript_id, GROUP_CONCAT(DISTINCT Protein_position) as positions"
					+", COUNT(DISTINCT Protein_position) as variation_count FROM dbsnp INNER JOIN convart_gene_to_db AS gdb ON gdb.db_id=Feature WHERE Protein_position > 0 "+
                     " GROUP BY gdb.convart_gene_id", con)

In [4]:
combined_data = pd.concat([clinvar_data, gnomad_data, cosmic_data, dbsnp_data], axis=0, ignore_index=True)
combined_data = combined_data.groupby('transcript_id').apply(lambda x: pd.Series({'positions':','.join([str(i) for i in x['positions']]), 
                                                                  'variation_count': sum(x['variation_count'])}))
combined_data = combined_data.reset_index()

In [5]:

variations = {
    'ptm': ptm_data,
	'variant': combined_data,
	'pathogenic': pathogenic_clinvar_data,
	'vus': vus_clinvar_data,
	'benign': benign_clinvar_data
}

for key, variation in variations.items():
	variation.loc[:, 'transcript_id'] = variation.loc[:, 'transcript_id'].astype(str) 
	variations[key] = variation.set_index('transcript_id', drop=False)

def conservation_scores_by_gene(row):
	scores = []

	ind, msa_details = row
   
	msa_details = msa_details.to_dict()
	msa_details['id'] = str(msa_details['id'])
    
	sequences = ('\n' + msa_details['fasta']).split('\n>')
	sequences = [{
			'transcript_id': msa_details['id'],
			'species_id': re.sub(r'^(.*?)\[([a-zA-Z0-9 ]+?)\]$', r'\2', seq.split('\n')[0]),
			'fasta': re.sub('^(.*?)\n', '', seq).replace('\n','').replace(' ', '')} for seq in sequences]
	sequences = list(filter(lambda x : x['fasta'].replace('\n','') !='', sequences))

	human_seq = sequences[0]['fasta']
	seq_length = len(human_seq)
	n_sequences = len(sequences)
	
	variation_positions_by_data = {}

	for key, data in variations.items():
		variation_positions_by_data[key] = set()
        
		if str(msa_details['id']) in data.index:
			cur_positions = data.loc[msa_details['id'], 'positions'].split(',')
			cur_positions = set([cur_position for cur_position in cur_positions ])
			[variation_positions_by_data[key].add(int(cur_position))  for cur_position in cur_positions if cur_position != '']
        
    
	for pairwise_id in range(1, len(sequences)):
		exact_match_score = 0
		polarity_match_score = 0
		
		exact_scores_by_data = {}
		polarity_scores_by_data = {}

		for key, data in variations.items():
			exact_scores_by_data[key] = 0
			polarity_scores_by_data[key] = 0

		pair = sequences[pairwise_id]
		pair_seq = pair['fasta']
		human_seq_position = 0
		
		for i in range(seq_length):
			if len(human_seq) != len(pair_seq):
				print(len(human_seq), len(pair_seq), msa_details['msa_id'])
				sys.exit(-1)
			if human_seq[i] == '-':
				continue
			human_seq_position += 1

			if human_seq[i] == 'X' or pair_seq[i] == 'X':
				continue

			aligned_aminoacids = 1 if len({seq[i] for seq in [human_seq, pair_seq]})==1 else 0
			exact_match_score += aligned_aminoacids

			aligned_aminoacids_polarity = 1 if  len({polarity[seq[i]] for seq in [human_seq, pair_seq]}) == 1 else 0
			polarity_match_score += aligned_aminoacids_polarity

			for key, positions in variation_positions_by_data.items():
				if human_seq_position not in positions:
					continue
				exact_scores_by_data[key] += aligned_aminoacids
				polarity_scores_by_data[key] += aligned_aminoacids_polarity

		scores.append({'transcript_id': msa_details['id'], 'pairwise_transcript_id': pair['transcript_id'],
						'pairwise_species_id': pair['species_id'],
						'score_type': 'exact_match_score', 'score': exact_match_score, 
						'number_of_aminoacids': len(msa_details['sequence']),
                      'msa_id': msa_details['msa_id']})
		#scores.append({'transcript_id': msa_details['id'], 'pairwise_transcript_id': pair['transcript_id'],
		#				'pairwise_species_id': pair['species_id'],
		#				'score_type': 'polarity_match_score', 'score': polarity_match_score,
		#				'number_of_aminoacids': len(msa_details['sequence'])})

		#Variant scores
		for key, exact_score in exact_scores_by_data.items():
			scores.append({'transcript_id': msa_details['id'], 'pairwise_transcript_id': pair['transcript_id'],
						'pairwise_species_id': pair['species_id'],
						'score_type': key+'_exact_match_score', 'score': exact_score, 
						'number_of_aminoacids': len(variation_positions_by_data[key]),
                      'msa_id': msa_details['msa_id']})
		#for key, polarity_score in polarity_scores_by_data.items():
		#	scores.append({'transcript_id': msa_details['id'], 'pairwise_transcript_id': pair['transcript_id'],
		#				'pairwise_species_id': pair['species_id'],
		#				'score_type': key+'_polarity_match_score', 'score': polarity_score, 
		#				'number_of_aminoacids': len(variation_positions_by_data[key])})

	return scores

#conservation_scores_by_gene(list(genes.iterrows())[0])
scores = imap_unordered_bar(conservation_scores_by_gene, genes.iterrows(), genes.shape[0], 30)


 29%|██▊       | 15871/55428 [00:07<00:17, 2290.52it/s]
15871it [00:07, 2281.30it/s]
 67%|██████▋   | 37201/55428 [00:17<00:08, 2050.02it/s]


 70%|██████▉   | 38791/55428 [00:18<00:06, 2393.15it/s]
38791it [00:18, 2323.54it/s]
 78%|███████▊  | 43372/55428 [00:20<00:05, 2200.66it/s]
43372it [00:20, 2158.48it/s]
 96%|█████████▌| 52951/55428 [00:25<00:01, 1700.37it/s]
52951it [00:24, 1628.86it/s]
 97%|█████████▋| 53851/55428 [00:25<00:00, 1908.91it/s]
53851it [00:25, 1835.71it/s]
100%|██████████| 55428/55428 [00:26<00:00, 2064.47it/s]


In [ ]:
df = pd.DataFrame(scores)
df

,msa_id,number_of_aminoacids,pairwise_species_id,pairwise_transcript_id,score,score_type,transcript_id
0,31,309,Mus musculus,371,269,exact_match_score,371
1,31,0,Mus musculus,371,0,ptm_exact_match_score,371
2,31,43,Mus musculus,371,38,variant_exact_match_score,371
3,31,0,Mus musculus,371,0,pathogenic_exact_match_score,371
4,31,0,Mus musculus,371,0,vus_exact_match_score,371
5,31,0,Mus musculus,371,0,benign_exact_match_score,371
6,31,309,Rattus norvegicus,371,269,exact_match_score,371
7,31,0,Rattus norvegicus,371,0,ptm_exact_match_score,371
8,31,43,Rattus norvegicus,371,38,variant_exact_match_score,371
9,31,0,Rattus norvegicus,371,0,pathogenic_exact_match_score,371


In [ ]:
df = pd.DataFrame(scores)

def get_id(x):
	global ncbi_to_transcript_id
	search_params = [x]

	for param in search_params:
		if param in ncbi_to_transcript_id.index:
			return ncbi_to_transcript_id.loc[param, 'gene_symbol']

	return None

df.loc[:, 'gene_symbol'] = df['transcript_id'].apply( get_id )

column_map = {
	'number_of_aminoacids': 'aminoacid_number',
	'pairwise_species_id': 'specie',
	'pairwise_transcript_id':'transcript_id_specie',
	'score': 'score',
	'score_type': 'score_type',
	'transcript_id': 'transcript_id',
	'gene_symbol': 'gene_symbol',
    'msa_id': 'msa_id'
}

df.columns = [column_map[col] for col in df.columns]
df.loc[df['gene_symbol'].isna(), 'gene_symbol'] = ''
cur.execute('TRUNCATE TABLE conservation_scores')
con.commit()
engine = create_engine("mysql+pymysql://{user}:{pw}@{server}/{db}"
                       .format(server=config['MYSQL_HOST'], user=config['MYSQL_USER'], pw=config['MYSQL_PASSWD'], 
                               db=config['MYSQL_DB']))

df.to_sql('conservation_scores', engine, if_exists='append', index=False)
